# Chatbot
by Muhammad Jaysy Ansharulloh
## Import datasets

In [ ]:
entities_filepath = 'datasets/entities.json'
intents_filepath = 'datasets/intents.json'

In [ ]:
import json

In [ ]:
with open(intents_filepath) as fi:
    intents = json.load(fi)

with open(entities_filepath) as fe:
    entities = json.load(fe)

In [ ]:
print(intents)

In [ ]:
print(entities)

## JSON to dataframe

In [ ]:
import pandas as pd

In [ ]:
df_intents = pd.json_normalize(intents)
raw_df_intents = df_intents

In [ ]:
df_intents

## Process Text
### download and import required libraries and dictionaries

In [ ]:
import nltk

language = 'english'
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet

stop_words = set(stopwords.words(language))

In [ ]:
import re

### text preprocessing

In [ ]:
intent_dictionary = {}
for index, row in df_intents.iterrows():
    if row['intent'] not in intent_dictionary:
        intent_dictionary[row['intent']] = []
    
    # regex remove non-word
    regex = r'[^\w]'

    # sentence tokenization
    sentences = nltk.sent_tokenize(re.sub(regex, ' ', row['text'].lower()))

    # word tokenization
    temp_words = [nltk.word_tokenize(sentence) for sentence in sentences]
    words = []
    for word in temp_words:
        # text lemmatization and stemming
        lemmatizer = WordNetLemmatizer()
        # stop words filter
        words.extend([lemmatizer.lemmatize(w, wordnet.VERB) for w in word if not w in stop_words])

    intent_dictionary[row['intent']].extend(words)

In [ ]:
intent_dictionary

### dictionary to json to dataframe

In [ ]:
json_intent = []
for key in intent_dictionary:    
    d = {}
    d['intent'] = key
    d['text'] = ' '.join(list(intent_dictionary[key]))
    json_intent.append(d)

In [ ]:
print(json_intent)

In [ ]:
df_intents = pd.DataFrame(json_intent)

In [ ]:
df_intents

## Scoring

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# list of text
texts = df_intents['text'].tolist()

In [ ]:
texts

### create model

In [ ]:
count_vectorizer = CountVectorizer()
intent_values = count_vectorizer.fit_transform(texts)

### model to dataframe

In [ ]:
feature_names = count_vectorizer.get_feature_names()
pd.DataFrame(intent_values.toarray(), columns=feature_names)

## Create ML Model

In [ ]:
X = intent_values.toarray()
y = df_intents.iloc[:, 0]

In [ ]:
print(X)
print(y)

### train test split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
print(X_train)
print(X_test)
print(y_train)
print(y_test)